In [1]:
import pandas as pd
from bokeh.sampledata.us_counties import data as counties
from bokeh.resources import CDN
from bokeh.embed import file_html

In [2]:
jobs = pd.read_csv('data\jobs_by_county_march_2019.csv', thousands=',')
housing = pd.read_csv('data\housing_by_county_2018.csv')
housing = housing[['Geographic Area Name','Estimate!!HOUSING OCCUPANCY!!Total housing units']].copy()
housing = housing.rename(columns={'Estimate!!HOUSING OCCUPANCY!!Total housing units':'housing_units'})
housing
jobs['Geographic Area Name'] = jobs['Area'] + ' County, Ohio'
jobs = jobs.rename(columns={'Employment March 2019':'jobs_march_2019'})
df = jobs.merge(housing,on='Geographic Area Name')
wage = []
for index, row in df.iterrows():
    w = float(row['Average weekly wage(1)'].replace(',',''))
    wage.append(w)
df['avg_weekly_wage'] = wage
df = df.drop(columns='Average weekly wage(1)')
df 

,Area,jobs_march_2019,Geographic Area Name,housing_units,avg_weekly_wage
0,Adams,5823,"Adams County, Ohio",12920,740.0
1,Allen,50378,"Allen County, Ohio",45063,923.0
2,Ashland,19123,"Ashland County, Ohio",22250,751.0
3,Ashtabula,30271,"Ashtabula County, Ohio",46107,728.0
4,Athens,21057,"Athens County, Ohio",26610,817.0
...,...,...,...,...,...
83,Washington,25062,"Washington County, Ohio",28236,915.0
84,Wayne,48486,"Wayne County, Ohio",46500,927.0
85,Williams,17380,"Williams County, Ohio",16660,804.0
86,Wood,67423,"Wood County, Ohio",53910,934.0


In [3]:
counties = {
    code: county for code, county in counties.items() if county["state"] == "oh"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]

lonlat = pd.DataFrame(list(zip(county_names, county_xs, county_ys)), columns =['Area', 'lat', 'lon']) 
df = df.merge(lonlat,on='Area')
df['unit_per_job'] = df['housing_units']/df['jobs_march_2019']
df

,Area,jobs_march_2019,Geographic Area Name,housing_units,avg_weekly_wage,lat,lon,unit_per_job
0,Adams,5823,"Adams County, Ohio",12920,740.0,"[-83.63654, -83.63855, -83.63981, -83.64034, -...","[38.66953, 38.66326, 38.65493, 38.65096, 38.64...",2.218788
1,Allen,50378,"Allen County, Ohio",45063,923.0,"[-84.33692, -84.31401, -84.28683, -84.25328, -...","[40.85913, 40.85941, 40.85964, 40.85975, 40.86...",0.894498
2,Ashland,19123,"Ashland County, Ohio",22250,751.0,"[-82.38847, -82.41706, -82.41737, -82.42082, -...","[40.80568, 40.81702, 40.82048, 40.82358, 40.82...",1.163520
3,Ashtabula,30271,"Ashtabula County, Ohio",46107,728.0,"[-81.00226, -81.00983, -81.00557, -81.00445, -...","[41.85256, 42.2516, 42.25229, 42.25231, 42.252...",1.523141
4,Athens,21057,"Athens County, Ohio",26610,817.0,"[-82.20324, -82.19434, -82.17204, -82.16717, -...","[39.46841, 39.46789, 39.46658, 39.46858, 39.47...",1.263713
...,...,...,...,...,...,...,...,...
83,Washington,25062,"Washington County, Ohio",28236,915.0,"[-81.09149, -81.08682, -81.0623, -81.04542, -8...","[39.57409, 39.57395, 39.57307, 39.57244, 39.57...",1.126646
84,Wayne,48486,"Wayne County, Ohio",46500,927.0,"[-82.04171, -82.06505, -82.08112, -82.09971, -...","[40.66835, 40.66837, 40.66819, 40.66809, 40.66...",0.959040
85,Williams,17380,"Williams County, Ohio",16660,804.0,"[-84.80403, -84.80404, -84.80404, -84.80406, -...","[41.43868, 41.44228, 41.44336, 41.44505, 41.44...",0.958573
86,Wood,67423,"Wood County, Ohio",53910,934.0,"[-83.49787, -83.4897, -83.45823, -83.41592, -8...","[41.61797, 41.61827, 41.61865, 41.61894, 41.61...",0.799579


In [4]:
df.to_csv('data\jobs_housing_clean.csv')

In [9]:
from bokeh.io import show
from bokeh.models import LogColorMapper
from bokeh.palettes import YlOrRd9 as palette
from bokeh.plotting import figure

#palette.reverse()
color_mapper = LogColorMapper(palette=palette)

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Occupied Housing Units per Job", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@Area")
       ,("Housing Units Per Job", "@unit_per_job")
       ,("Employment (March 2019)", "@jobs_march_2019")
       ,("Housing Units (2018)", "@housing_units")
             ])

p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('lat', 'lon', source=df,
          fill_color={'field': 'unit_per_job', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

GlyphRenderer(id='1128', ...)

In [10]:
show(p)

In [11]:
html = file_html(p, CDN, "jobs per housing unit - Ohio")

In [12]:
f= open("jobs_per_housing_unit.txt","w+")
f.write(html)
f.close()